In [ ]:
import warnings

from evo.notebooks import ServiceManagerWidget

# Suppress all warnings
warnings.simplefilter("ignore")

# Evo app credentials
client_id = "<your-client-id>"  # Replace with your client ID
redirect_url = "<your-redirect-url>"  # Replace with your redirect URL

manager = ServiceManagerWidget.with_auth_code(
    redirect_url=redirect_url,
    client_id=client_id,
)

In [ ]:
from helpers.widgets import create_duf_widget

ui = await create_duf_widget(manager)
display(ui)